In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import cv2
import torch
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torchvision
import shutil
import imageio
from IPython import display
from PIL import Image
from glob import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid, save_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])

In [ ]:
data = torchvision.datasets.MNIST(root='./mnist', train=True, download=True, transform=transform)

In [ ]:
batch_size = 100
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [ ]:
z_dim = 100
image_dim = 28 * 28
num_classes = 10

# Visualization

In [ ]:
data_iter = iter(dataloader)
images, _ = next(data_iter)

img_grid = make_grid(images, nrow=10)

plt.figure(figsize=(8, 8))
plt.imshow(img_grid.numpy()[0], cmap='gray')
plt.axis('off')
plt.show()

# Generator Network

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.generator(x)

In [ ]:
generator = Generator(input_dim=z_dim, output_dim=image_dim).to(device)

# Discriminator Network

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.discriminator(x)

In [ ]:
discriminator = Discriminator(input_dim=image_dim).to(device)

# Train

In [ ]:
lr = 0.0002
criterion = nn.BCELoss()
generator_optimizer = optim.Adam(generator.parameters(), lr=lr)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr)

In [ ]:
d_loss_s = []
g_loss_s = []

In [ ]:
os.makedirs('lgan')

In [ ]:
n_epochs = 200

In [ ]:
for epoch in range(n_epochs):
    for batch_idx, (images, labels) in enumerate(dataloader):
        discriminator.zero_grad()

        x_real, y_real = images.view(-1, image_dim), torch.full((batch_size, 1), 0.9)
        x_real, y_real = x_real.to(device), y_real.to(device)

        discriminator_output = discriminator(x_real)
        discriminator_real_loss = criterion(discriminator_output, y_real)

        z = torch.randn(batch_size, z_dim).to(device)
        x_fake, y_fake = generator(z), torch.full((batch_size, 1), 0.1).to(device)

        discriminator_output = discriminator(x_fake)
        discriminator_fake_loss = criterion(discriminator_output, y_fake)

        discriminator_loss = discriminator_real_loss + discriminator_fake_loss
        
        discriminator_loss.backward()
        discriminator_optimizer.step()

        d_loss_s.append(discriminator_loss.item())

        for param in discriminator.parameters():
            param.data.clamp_(-0.01, 0.01)

        if batch_idx % 2 == 0:
            generator.zero_grad()
    
            z = torch.randn(batch_size, z_dim).to(device)
            y = torch.full((batch_size, 1), 0.9).to(device)
    
            generator_output = generator(z)
            discriminator_output = discriminator(generator_output)
            generator_loss = criterion(discriminator_output, y)
    
            generator_loss.backward()
            generator_optimizer.step()

            g_loss_s.append(generator_loss.item())

    print(f'[{epoch}/{n_epochs}], Loss_D: {discriminator_loss.item()}, Loss_G: {generator_loss.item()}')

    test_z = torch.randn(10, z_dim).to(device)
    generated = generator(test_z)
    save_image(generated.view(generated.size(0), 1, 28, 28), f'./lgan/sample_images_{epoch}.png')

# Results

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(g_loss_s)
plt.plot(d_loss_s)
plt.title("Loss Curve")
plt.legend(["Generator Loss", "Discriminator Loss"])
plt.show()

# GIF

In [ ]:
files = glob("./lgan/*.png")
gif_file = "./lgan/lb_gan.gif"

In [ ]:
with imageio.get_writer(gif_file, mode='I') as writer:
    for file in files:
        image = imageio.imread(file)
        writer.append_data(image)

In [ ]:
display.Image(filename=gif_file)